In [3]:
import string
import re
import time


In [205]:
regex = re.compile('[%s]' % re.escape(string.punctuation))

def re_remove_punct(s):  # From Vinko's solution, with fix.
    return regex.sub('', s)

def shingles(s, k, res = 'result ' + time.strftime("%Y%m%d-%H%M%S") + '.txt'):
    f = open(res, 'a')
    while True:
        result = (s[:k], s[1:])
        s = result[1]
        if result[0] == '':
            break
        f.write(result[0] + '\n')
    f.close()

f = open('wasteland.txt', 'r')
waste = f.read()
f.close()

waste = waste.replace('\n',' ')
waste = waste.replace('“', '')
waste = waste.replace('”', '')
waste = waste.replace('’', '')
waste = re_remove_punct(waste)
waste = waste.lower()
waste

waste_list = []

test = 'a bc defg hij k && lmno pqr stuv wxy z'  
        
# shingles(waste, 9, 'waste_result.txt')



In [227]:
f = open('wasteland.txt', 'r')
waste = f.read()
f.close()

waste = waste.replace('“', '')
waste = waste.replace('”', '')
waste = waste.replace('’', '')
waste = re_remove_punct(waste)
waste = waste.lower()

f = open('wasteland2.txt', 'r')
waste2 = f.read()
f.close()

waste2 = waste2.replace('“', '')
waste2 = waste2.replace('”', '')
waste2 = waste2.replace('’', '')
waste2 = re_remove_punct(waste2)
waste2 = waste2.lower()

# print(waste2[:1000])

print(Simhash(waste).distance(Simhash(waste2)))
print(Simhash('1').distance(Simhash('2')))

8
29


In [118]:
print(Simhash("a bc defg hij k && lmno pqr stuv wxy z").distance(Simhash("a bc defg hij k && lmno pqr stuv wxy zaaaaaaaa")))


18


In [230]:
# print(int('001000001',2))
# print(1<<2)
# def bin(s):
#     return str(s) if s<=1 else bin(s>>1) + str(s&1)

# print(bin(12))
# import hashlib

# hashed = hashlib.md5("shingle".encode('utf-8')).hexdigest()
# hashed2 = hashlib.md5("shingles".encode('utf-8')).hexdigest()
# print(hashed)
# print(hashed2)
# print(' '.join(map(bin,bytearray(hashed,'utf8'))))
# print()
# print(' '.join(map(bin,bytearray(hashed2,'utf8'))))
# int(hashed2,16)
# int(hashlib.md5('yoyo'.encode('utf-8')).hexdigest(), 16)


import os
import hashlib

array = os.urandom(1 << 20)
# print(array)
md5 = hashlib.md5()
md5.update(array)
digest = md5.hexdigest()
print(digest)
number = int(digest, 16)
print(number)

print(number % 1)
6 ^ 2

e77e5518405e7c6593c285c843c26c48
307707622356704432759126517609369070664
0


4

In [155]:
import re
# from simhash import Simhash, SimhashIndex
def get_features(s):
    width = 3
    s = s.lower()
    s = re.sub(r'[^\w]+', '', s)
    return [s[i:i + width] for i in range(max(len(s) - width + 1, 1))]

data = {
    1: u'How are you? I Am fine. blar blar blar blar blar Thanks.',
    2: u'How are you i am fine. blar blar blar blar blar than',
    3: u'This is simhash test.',
}

# print([(str(index), string) for index, string in data.items()])

objs = [(str(index), Simhash(get_features(string))) for index, string in data.items()]
print(objs) # list of tuples where the tuples are the index of the dictionary objs and the values are the SimHash'd objects
index = SimhashIndex(objs, k=3)

print("index bucket size:", index.bucket_size())

s1 = Simhash(get_features(u'How are you i am fine. blar blar blar blar blar thank'))
print("index.get near dups:", index.get_near_dups(s1))

index.add('2', s1)
print("index.get near dups:", index.get_near_dups(s1))
print('index.get_keys(s1):', list(index.get_keys(s1)))

#what is "get_keys(s1)" returning?
# print('offset:', index.offsets)
# print('f:', index.f)

# m = 2**index.offsets[1]-index.offsets[0]-1
# print(m)
# print(index.offsets[0])
# print(index.offsets[0] & m)
# m = 2** (4-48)-1
# print(index.offsets[3] & m)
print(list(index.get_keys(s1))[2])
print(index.bucket[list(index.get_keys(s1))[3]])
# print(get_features(data[1]))
# sim2, obj_id = 
dups = index.bucket[list(index.get_keys(s1))[3]]
for dup in dups:
    sim2, obj_id = dup.split(',',1)
    break
sim2



[('1', <__main__.Simhash object at 0x00000180C83D2B70>), ('2', <__main__.Simhash object at 0x00000180C83D2B38>), ('3', <__main__.Simhash object at 0x00000180C83D2668>)]
index bucket size: 11
index.get near dups: ['1']
index.get near dups: ['2', '1']
index.get_keys(s1): ['7a14: 0', '91c3: 1', 'e5cd: 2', '6988: 3']
e5cd: 2
{'6988e5cd91837a14,1', '6988e5cd91c37a14,2'}


'6988e5cd91837a14'

# SimHash Code (Shift-E to unhide)

In [112]:
# Created by 1e0n in 2013
from __future__ import division, unicode_literals

import sys
import re
import hashlib
import logging
import collections
from itertools import groupby

if sys.version_info[0] >= 3:
    basestring = str
    unicode = str
    long = int
else:
    range = xrange


def _hashfunc(x):
    return int(hashlib.md5(x).hexdigest(), 16)


class Simhash(object):

    def __init__(self, value, f=64, reg=r'[\w\u4e00-\u9fcc]+', hashfunc=None):
        """
        `f` is the dimensions of fingerprints
        `reg` is meaningful only when `value` is basestring and describes
        what is considered to be a letter inside parsed string. Regexp
        object can also be specified (some attempt to handle any letters
        is to specify reg=re.compile(r'\w', re.UNICODE))
        `hashfunc` accepts a utf-8 encoded string and returns a unsigned
        integer in at least `f` bits.
        """

        self.f = f
        self.reg = reg
        self.value = None

        if hashfunc is None:
            self.hashfunc = _hashfunc
        else:
            self.hashfunc = hashfunc

        if isinstance(value, Simhash):
            self.value = value.value
        elif isinstance(value, basestring):
            self.build_by_text(unicode(value))
        elif isinstance(value, collections.Iterable):
            self.build_by_features(value)
        elif isinstance(value, long):
            self.value = value
        else:
            raise Exception('Bad parameter with type {}'.format(type(value)))

    def _slide(self, content, width=4):
        return [content[i:i + width] for i in range(max(len(content) - width + 1, 1))]

    def _tokenize(self, content):
        content = content.lower()
        content = ''.join(re.findall(self.reg, content))
        ans = self._slide(content)
        return ans

    def build_by_text(self, content):
        features = self._tokenize(content)
        features = {k:sum(1 for _ in g) for k, g in groupby(sorted(features))}
        return self.build_by_features(features)

    def build_by_features(self, features):
        """
        `features` might be a list of unweighted tokens (a weight of 1
                   will be assumed), a list of (token, weight) tuples or
                   a token -> weight dict.
        """
        v = [0] * self.f
        masks = [1 << i for i in range(self.f)]
        if isinstance(features, dict):
            features = features.items()
        for f in features:
            if isinstance(f, basestring):
                h = self.hashfunc(f.encode('utf-8'))
                w = 1
            else:
                assert isinstance(f, collections.Iterable)
                h = self.hashfunc(f[0].encode('utf-8'))
                w = f[1]
            for i in range(self.f):
                v[i] += w if h & masks[i] else -w
        ans = 0
        for i in range(self.f):
            if v[i] > 0:
                ans |= masks[i]
        self.value = ans

    def distance(self, another):
        assert self.f == another.f
        x = (self.value ^ another.value) & ((1 << self.f) - 1)
        ans = 0
        while x:
            ans += 1
            x &= x - 1
        return ans


class SimhashIndex(object):

    def __init__(self, objs, f=64, k=2):
        """
        `objs` is a list of (obj_id, simhash)
        obj_id is a string, simhash is an instance of Simhash
        `f` is the same with the one for Simhash
        `k` is the tolerance
        """
        self.k = k
        self.f = f
        count = len(objs)
        logging.info('Initializing %s data.', count)

        self.bucket = collections.defaultdict(set)

        for i, q in enumerate(objs):
            if i % 10000 == 0 or i == count - 1:
                logging.info('%s/%s', i + 1, count)

            self.add(*q)

    def get_near_dups(self, simhash):
        """
        `simhash` is an instance of Simhash
        return a list of obj_id, which is in type of str
        """
        assert simhash.f == self.f

        ans = set()

        for key in self.get_keys(simhash):
            dups = self.bucket[key]
            logging.debug('key:%s', key)
            if len(dups) > 200:
                logging.warning('Big bucket found. key:%s, len:%s', key, len(dups))

            for dup in dups:
                sim2, obj_id = dup.split(',', 1)
                sim2 = Simhash(long(sim2, 16), self.f)

                d = simhash.distance(sim2)
                if d <= self.k:
                    ans.add(obj_id)
        return list(ans)

    def add(self, obj_id, simhash):
        """
        `obj_id` is a string
        `simhash` is an instance of Simhash
        """
        assert simhash.f == self.f

        for key in self.get_keys(simhash):
            v = '%x,%s' % (simhash.value, obj_id)
            self.bucket[key].add(v)

    def delete(self, obj_id, simhash):
        """
        `obj_id` is a string
        `simhash` is an instance of Simhash
        """
        assert simhash.f == self.f

        for key in self.get_keys(simhash):
            v = '%x,%s' % (simhash.value, obj_id)
            if v in self.bucket[key]:
                self.bucket[key].remove(v)

    @property
    def offsets(self):
        """
        You may optimize this method according to <http://www.wwwconference.org/www2007/papers/paper215.pdf>
        """
        return [self.f // (self.k + 1) * i for i in range(self.k + 1)]

    def get_keys(self, simhash):
        for i, offset in enumerate(self.offsets):
            if i == (len(self.offsets) - 1):
                m = 2 ** (self.f - offset) - 1
            else:
                m = 2 ** (self.offsets[i + 1] - offset) - 1
            c = simhash.value >> offset & m
            yield '%x: %x' % (c, i)

    def bucket_size(self):
        return len(self.bucket)
    
    def print_objs(self):
        return None
    
    def print_buckets(self):
        return self.bucket
        
    def print_f(self):
        return self.f	

### End of SimHash Code

# Pre-Processing Code (Shift-E to unhide)

In [40]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
test = 'a bc defg hij k && lmno pqr stuv \t wxy z \n abcd nate is the  ;best  '  

def re_remove_punct(s):  # From Vinko's solution, with fix.
    return regex.sub('', s)

def remove_spaces(s, extra = False):
    s = s.lower()
    s = re_remove_punct(s)
    s = " ".join(s.split())
    if extra == True:
        s = s.replace(' ','')
    return s

def create_shingled_features(s, k, clean = False, remove_all_spaces= False, print_to_file = False, res = 'result ' + time.strftime("%Y%m%d-%H%M%S") + '.txt'):
    output = []
    if clean == True:
        s = remove_spaces(s, extra = remove_all_spaces)
    f = open(res, 'a')
    while True:
        result = (s[:k], s[1:])
        s = result[1]
        if result[0] == '':
            break
        if print_to_file == True:
            f.write(result[0] + '\n')
        else:
            output.append(result[0])
    f.close()
    if print_to_file == True:
        print("File printed to '" + res + "'")
    else:
        return output


In [176]:
#### import Shari's plagiarized samples

import glob
path="C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\corpus-20090418\\*.txt"
chop = "C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\corpus-20090418\\"

plag = {} #store Shari's sample plagiarism files here

for filename in glob.iglob(path):
    chopt = filename.replace(chop,'')
    with open(filename, encoding="ISO-8859-1") as f:
        plag[chopt] = f.read()
    
keys = list(plag.keys())

### Create the index of simhashed files

objs = [(str(k), Simhash(create_shingled_features(s, 3, clean=True))) for k, s in plag.items()]
index = SimhashIndex(objs, k=3)

simhasht_plags = {}

for i in objs:
    simhasht_plags[i[0]] = i[1]


In [177]:
#import and simhash the original files

import glob
path="C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\originals\\*.txt"
chop = "C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\originals\\"

originals = {} #store Shari's sample original files here
simhasht = {}

for filename in glob.iglob(path):
    chopt = filename.replace(chop,'')
    with open(filename, encoding="ISO-8859-1") as f:
        originals[chopt] = f.read()
    simhasht[chopt] = Simhash(create_shingled_features(originals[chopt], 3, clean=True))
    
keys = list(originals.keys())


# simhasht['orig_taska.txt']

In [179]:
index.get_near_dups(simhasht['orig_taska.txt'])
# simhasht_plags['g0pE_taska.txt'].distance(simhasht['orig_taska.txt'])


['g0pE_taska.txt']

In [100]:
# filename = 'C:\\Users\\nathan.a.miller\\Documents\\Jupyter\\Python\\g0pA_nate_taska.txt'
path="C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\corpus-20090418\\*.txt"
chop = "C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\corpus-20090418\\"

with open(filename, encoding="ISO-8859-1") as f:
        s1 = f.read()
        
s1 = Simhash(create_shingled_features(s1, 3, clean=True))

print(index.bucket_size())
print(index.get_near_dups(s1))
l = index.get_keys(s1)


328
['g0pA_taska.txt']


In [187]:
### Create hamming distance matrix

#### import Shari's plagiarized samples

import glob
path="C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\o+p\\*.txt"
chop = "C:\\Users\\nathan.a.miller\\OneDrive - Accenture Federal Services\\USCIS\\Text\\Plagiarism Sample Text\\o+p\\"

all_dict = {} #store Shari's sample plagiarism files here

for filename in glob.iglob(path):
    chopt = filename.replace(chop,'')
    with open(filename, encoding="ISO-8859-1") as f:
        all_dict[chopt] = f.read()
    
keys = list(all_dict.keys())

### Create the index of simhashed files

objs = [(str(k), Simhash(create_shingled_features(s, 3, clean=True))) for k, s in plag.items()]
index = SimhashIndex(objs, k=3)

all_simhasht = {}

for i in objs:
    all_simhasht[i[0]] = i[1]

In [188]:
len(all_simhasht)

95